In [24]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

load_dotenv()

llama_api_key = os.getenv('LLAMA_API_KEY')


model_name = "llama-3.1-8b-instant"
parser = StrOutputParser()

model = ChatGroq(api_key=llama_api_key, model=model_name)

chain = model | parser

print(chain.invoke("when was the eifel tower built "))

The Eiffel Tower was built for the 1889 World's Fair in Paris, France. Construction began in January 1887 and was completed in March 1889. The tower was officially opened to the public on May 15, 1889. It was designed and built by the French engineering company Compagnie des Établissements Eiffel, led by Gustave Eiffel.


In [3]:
from langchain_openai.chat_models.azure import AzureChatOpenAI
import os
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
gpt_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

index_path = "../api/index/faiss_index"

llm = AzureChatOpenAI(
                openai_api_version=openai_api_version,
                azure_deployment=gpt_model,
            )

embeddings = AzureOpenAIEmbeddings(azure_endpoint=openai_api_base,
                api_key=openai_api_key,
                azure_deployment=embeddings_model)

In [ ]:
# define model to use


In [25]:
#@tool
def rag(query:str):
        """Use this to execute RAG. If the question is related to gen ai in art or music, using this tool retrieve the results."""
        print("Calling RAG...")
        vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
        
        retriever = vector_store.as_retriever()
    
        template = """Answer questions based only on the following context: 
        
        ### CONTEXT ###
        {context}
        
        Question: {question}
        
        If the context doesn't provide enough information to answer the question, say that you don't know.
        Do not respond with information outside the previous context."""

        prompt = PromptTemplate.from_template(template)

        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | model
            | StrOutputParser()
        )

        result = chain.invoke(query)
        return result

In [36]:
import wikipedia

def wikipedia_search_tool(query: str, sentences: int = 1):
    """Searchs content in the wikipedia web site"""
    
    return wikipedia.summary(query, sentences = sentences)

In [29]:
def generate_descriptive_text(context: str, query: str):
    """This tool analyzes context and user query and returns a very good prompt to generate images"""
    template = """Your job is to analyze the following context:
    ### CONTEXT ###
    {context}

    Generate a detailed prompt to generate an image based on the user query and the context.

    Query: {query}
    """

    prompt = PromptTemplate.from_template(template)

    chain = (
            prompt | model | StrOutputParser()
    )

    result = chain.invoke({"context" :context, "query": query})
    return result



In [30]:
from openai import AzureOpenAI
from datetime import datetime
import requests
import json
import os
from uuid import uuid4

dalle_model = os.getenv('GPT_IMAGES_DEPLOYMENT_NAME')
images_folder = os.getcwd() + "\\assets\\generated_images"

client = AzureOpenAI(azure_endpoint=openai_api_base, 
                     api_key=openai_api_key, 
                     api_version=openai_api_version)

@tool
def generate_images(prompt: str):
        """Generate images, diagrams, charts, etc., based on prompts"""

        sub_folder = (datetime.today().strftime('%Y-%m-%d %H:%M:%S')).replace(" ","-").replace(":","-")
        images_path = os.path.join(images_folder, sub_folder)

        if not os.path.exists(images_path):
            os.makedirs(images_path) 

        result = client.images.generate(
                    model = dalle_model,
                    prompt=prompt,
                    n=1
                    )
        
        image_url = json.loads(result.model_dump_json())['data'][0]['url']
        
        image = requests.get(image_url).content
        image_name = f"genai_img_{uuid4()}"

        with open(os.path.join(images_folder, images_path, f"{image_name}.jpg"), 'wb') as handler:
            handler.write(image)
        
        return image_url

In [ ]:
# two teams
# search team, visualization team (images) or docs
research_agent = create_react_agent(
    model = model,
    tools=[wikipedia_search_tool],
    name = "research_expert",
    prompt = "You are a world class researcher with access to wikipedia, return the least amount of words you can"
)

rag_agent = create_react_agent(
    model = model,
    tools=[rag],
    name = "rag_expert",
    prompt = "You are a RAG tool with access to a vector DB with GEN AI in ART and Music Documents"
)

search_team = create_supervisor(
    [research_agent, rag_agent],
    model = model,
    prompt = (
        "You are a team supervisor managing a search expert and a RAG search expert" \
        "For current events and information, use the research_agent" \
        "For queries related to Gen AI in art, Gen AI in Music, use the rag_agent"
    )
).compile(name="search_team")

In [32]:
def summary(input:str):
    "creates a summary of the content"
    prompt = f"review the given content {input}, and create a brief summary"
    return model.invoke(prompt)

summary_agent = create_react_agent(
    model = model,
    tools = [summary],
    name = "summary_expert",
    prompt = "You are a world class agent that creates smart summaries of text"
)

def writer(text_to_dump: str):
    "writes in txt file"
    with open("final_test.txt", "w") as f:
        f.write(text_to_dump)
    print("Text has been written to final_test.txt")


writing_agent = create_react_agent(
    model = model,
    tools = [writer],
    name = "writing_expert",
    prompt = "Your job is to write the final content in a text file"
)

publishing_team = create_supervisor(
    [summary_agent, writing_agent],
    model = model,
    prompt= ("You are a team supervisor managing summaring experte and writing expert" \
    "For creating summaries of text, use summary_agent" \
    "For writing final content in a text file, make sure you only use writing_agent")
).compile(name="publishing_team")

In [38]:
top_level_supervisor = create_supervisor(
    [search_team, publishing_team],
    model = model,
    prompt = ("You are a top level supervisor managing a search team and publishing team." \
    "For recent envents and information, use search_team" \
    "For summaring and writing final content in a text file, use publishing team." \
    "Always do handoff to the publishing_team and use writing_agent in the publishing_team to write into a text file"),
    output_mode="full_history"
).compile(name="top_level_supervisor")

In [41]:
result =top_level_supervisor.invoke(
    {
        "messages":[
            {
                "role": "user",
                "content": "Tell me about the making art with gen ai"
            }
        ]
    }
)

Calling RAG...
Calling RAG...
Calling RAG...
Calling RAG...
Calling RAG...


KeyboardInterrupt: 

In [40]:
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

Tell me about the concacaf champions league
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_search_team (call_mx1d)
 Call ID: call_mx1d
  Args:
================================= Tool Message =================================
Name: transfer_to_search_team

Successfully transferred to search_team
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  wikipedia_search (call_gpgg)
 Call ID: call_gpgg
  Args:
    search_term: concacaf champions league
================================= Tool Message =================================
Name: wikipedia_search

Error: wikipedia_search is not a valid tool, try one of [transfer_to_research_expert, transfer_to_rag_expert].
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transf